In [ ]:
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data
max_steps = 200#1000
learning_rate = 0.001
dropout = 0.9
data_dir = "../../data/mnsit/"
log_dir = "./log"

mnist = input_data.read_data_sets(data_dir,one_hot=True)

#sess = tf.InteractiveSession

with tf.name_scope('input'):
    x = tf.placeholder(tf.float32, [None,784], name='x-input')
    y_ = tf.placeholder(tf.float32, [None,10], name='x-input')
with tf.name_scope('input_reshape'):
    image_shaped_input = tf.reshape(x,[-1,28,28,1])
    tf.summary.image('input', image_shaped_input, 10)#10

global_step = tf.Variable(0, trainable=False)
    
def weight_variable(shape):
    initial = tf.truncated_normal(shape , stddev=0.1)
    return tf.Variable(initial)

def bias_variable(shape):
    initial = tf.constant(0.1,shape=shape)
    return tf.Variable(initial)

def variable_summaries(var):
    with tf.name_scope('summaries'):
        mean = tf.reduce_mean(var)
        # tf中标量(scalar)监控数据随着迭代进行的变化趋势
        tf.summary.scalar('mean',mean)
        with tf.name_scope('stddev'):
            stddev = tf.sqrt(tf.reduce_mean(tf.square(var - mean)))
        tf.summary.scalar('stddev',stddev)
        tf.summary.scalar('max',tf.reduce_max(var))
        tf.summary.scalar('min',tf.reduce_min(var))
        # tf中张量分布监控数据随着迭代论数的变化趋势
        # 神经网络参数取值分布随着迭代进行的变化趋势
        tf.summary.histogram('histogram', var)
        
def nn_layer(input_tensor,input_dim,output_dim,layer_name,act=tf.nn.relu):
    with tf.name_scope(layer_name):
        with tf.name_scope('weight'):
            weights = weight_variable([input_dim,output_dim])
            variable_summaries(weights)
        with tf.name_scope('bias'):
            biases = bias_variable([output_dim])
            variable_summaries(biases)
        with tf.name_scope('Wx_plus_b'):
            preactivate = tf.matmul(input_tensor,weights) + biases
            tf.summary.histogram('pre_activations',preactivate)
            
                        
        activations = act(preactivate,name='activations')
        tf.summary.histogram('activations',activations)
        return activations
    
hidden1 = nn_layer(x, 784, 500, 'layer1')
with tf.name_scope('dropout'):
    keep_prob = tf.placeholder(tf.float32) 
    tf.summary.scalar('dropout_keep_probablility',keep_prob)
    dropped = tf.nn.dropout(hidden1,keep_prob)
    
y = nn_layer(dropped, 500, 10, 'layer2',act=tf.identity)


with tf.name_scope('cross_entropy'):
    diff = tf.nn.softmax_cross_entropy_with_logits(logits=y, labels=y_)
    with tf.name_scope('total1'):
        cross_entroy = tf.reduce_mean(diff)
tf.summary.scalar('cross_entropy',cross_entroy)

with tf.name_scope('train_step'):
    learning_rate_exp = tf.train.exponential_decay(
        learning_rate,
        global_step,
        120,
        0.9,
        staircase=True
    )

with tf.name_scope('train'):
    train_step = tf.train.AdamOptimizer(learning_rate_exp).minimize(cross_entroy, global_step=global_step)

with tf.name_scope('accuracy'):
    with tf.name_scope('correct_prediction'):
        correct_prediction = tf.equal(tf.argmax(y,1),tf.argmax(y_,1))
    with tf.name_scope('accuracy'):
        accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
        
tf.summary.scalar('accuracy',accuracy)


merged = tf.summary.merge_all()
init = tf.initialize_all_variables()
sess = tf.Session()
with sess.as_default():
    train_writer = tf.summary.FileWriter(log_dir + '/train')
    test_writer = tf.summary.FileWriter(log_dir + '/test')
    tf.global_variables_initializer().run()


def feed_dict(train):
    if(train):
        xs,ys = mnist.train.next_batch(100)
        k = dropout
    else:
        xs,ys = mnist.test.images,mnist.test.labels
        k = 1.0
    return {x:xs,y_:ys,keep_prob:k}

saver = tf.train.Saver()
for i in range(max_steps):
    if i % 10 == 0:
        summary,acc = sess.run([merged,accuracy],feed_dict=feed_dict(False))
        test_writer.add_summary(summary,i)
        print('Accuracy at step %s:%s'%(i,acc))
    else:
        if i %100 == 99:
            # 配置运行时需要记录的信息
            run_options = tf.RunOptions(trace_level=tf.RunOptions.FULL_TRACE)
            # 运行时记录运行信息的proto
            run_metadata = tf.RunMetadata()
            # 将配置信息和记录运行信息的proto传入运行的过程，
            # 从而记录运行时每一个节点的时间、空间开销信息
            summary,_ = sess.run([merged,train_step],feed_dict=feed_dict(True),
                                 options=run_options,run_metadata=run_metadata)
            # 将节点在运行时的信息写入日志文件
            train_writer.add_run_metadata(run_metadata, 'step%03d' % i)
            train_writer.add_summary(summary, i)
            saver.save(sess,log_dir+"/model.ckpt",i)
            print('Adding run metadata for',i)
        else:
            summary, _ = sess.run([merged,train_step],feed_dict=feed_dict(True))
            train_writer.add_summary(summary,i)
            
            
train_writer.close()
test_writer.close()  

E:\Anaconda3\tensorflow\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Please write your own downloading logic.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting ../../data/mnsit/train-images-idx3-ubyte.gz
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting ../../data/mnsit/train-labels-idx1-ubyte.gz
Instructions for updating:
Please use tf.one_hot on tensors.
Extracting ../../data/mnsit/t10k-images-idx3-ubyte.gz
Extracting ../../data/mnsit/t10k-labels-idx1-ubyte.gz
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See @{tf.nn.softmax_cross_entropy_with_logits_v2}.

Instructions for updating:
Use `tf.global_variables_initializer` instead.
Accuracy at

In [ ]:
ummary_waiter = tf.summary.FileWriter(log_dir + '/train',tf.get_default_graph())
ummary_waiter.close()